In [ ]:
import os
from google.colab import drive
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
drive.mount("/gdrive", force_remount=True)
main_dir = "/gdrive/MyDrive/ITCS6156_BERT"
result_dir = main_dir + "/Results"
train_path = main_dir + "/Data/train_data.txt"
test_path = main_dir + "/Data/test_data_solution.txt"

!ls {train_path}
!ls {test_path}

Mounted at /gdrive
ls: cannot access '/gdrive/MyDrive/ITCS6156_BERT/Data/train_data.txt': No such file or directory
ls: cannot access '/gdrive/MyDrive/ITCS6156_BERT/Data/test_data_solution.txt': No such file or directory


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize

#!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import torch
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, AutoConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#pip install transformers==4.45.2 sentence-transformers==3.1.1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenizer_function(df):
    tokenized = tokenizer(df['text'], padding=True, truncation=True)
    tokenized['labels'] = df['labels']

    return tokenized

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=1)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
class CustomTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # self.class_weights = class_weights
        self.class_weights = class_weights.to(self.model.device)

    def compute_loss(self, model, inputs, return_outputs=False):
        # labels = inputs.get("labels")
        labels = inputs.get("labels").to(self.model.device)
        # outputs = model(**inputs)
        outputs = model(**{k: v.to(self.model.device) for k, v in inputs.items()})
        logits = outputs.get("logits")

        loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# 1. Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Load data
train_path = '/content/drive/MyDrive/ITCS6156_BERT/Data/train_data.txt'

train_data = pd.read_csv(train_path, sep=':::', names=['Title', 'Genre', 'Description'], engine='python')
# test_data = pd.read_csv(test_path, sep=':::', names=['Id', 'Title', 'Genre', 'Description'], engine='python')
# test_data.drop(columns=["Id"], inplace=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ITCS6156_BERT/Data/train_data.txt'

In [ ]:
train_data

NameError: name 'train_data' is not defined

In [ ]:
train_data['Genre'].unique()

In [ ]:
genre_counts = train_data['Genre'].value_counts()
genre_counts

# 2. Data pre-processing

In [ ]:
processed_train_data = train_data.rename(columns={'Description': 'text'})
processed_train_data['text'] = processed_train_data['text'].apply(lambda x: str(x).replace("\xa0", "").lower())
processed_train_data['text'] = processed_train_data['text'].apply(lambda x: x.replace('\n', ' ').replace('\r', ' '))
processed_train_data=processed_train_data.drop_duplicates()

# processed_test_data = test_data.rename(columns={'Description': 'text'})
# processed_test_data['text'] = processed_test_data['text'].apply(lambda x: str(x).replace("\xa0", "").lower())
# processed_test_data['text'] = processed_test_data['text'].apply(lambda x: x.replace('\n', ' ').replace('\r', ' '))
# processed_test_data=processed_test_data.drop_duplicates()

In [ ]:
# Convert text label to numeric label
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
processed_train_data['labels'] = label_encoder.fit_transform(processed_train_data['Genre'])
# processed_test_data['labels'] = label_encoder.fit_transform(processed_test_data['Genre'])
count_label = len(label_encoder.classes_)

In [ ]:
processed_train_data

,Title,Genre,text,labels
1,Oscar et la dame rose (2009),drama,listening in to a conversation between his do...,8
2,Cupid (1997),thriller,a brother and sister with a past incestuous r...,24
3,"Young, Wild and Wonderful (1980)",adult,as the bus empties the students for their fie...,1
4,The Secret Sin (1915),drama,to help their unemployed father make ends mee...,8
5,The Unrecovered (2007),drama,the film's title refers not only to the un-re...,8
...,...,...,...,...
54210,"""Bonino"" (1953)",comedy,this short-lived nbc live sitcom centered on ...,5
54211,Dead Girls Don't Cry (????),horror,the next generation of exploitation. the sist...,13
54212,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"ze bestaan echt, is a stand-up comedy about g...",7
54213,Make Your Own Bed (1944),comedy,walter and vivian live in the country and hav...,5


In [ ]:
# processed_test_data

,Title,Genre,text,labels
0,Edgar's Lunch (1998),thriller,"l.r. brane loves his life - his car, his apar...",24
1,La guerra de papá (1977),comedy,"spain, march 1964: quico is a very naughty ch...",5
2,Off the Beaten Track (2010),documentary,one year in the life of albin and his family ...,7
3,Meu Amigo Hindu (2015),drama,"his father has died, he hasn't spoken with hi...",8
4,Er nu zhai (1955),drama,before he was known internationally as a mart...,8
...,...,...,...,...
54195,"""Tales of Light & Dark"" (2013)",horror,"covering multiple genres, tales of light & da...",13
54196,Der letzte Mohikaner (1965),western,as alice and cora munro attempt to find their...,26
54197,Oliver Twink (2007),adult,a movie 169 years in the making. oliver twist...,1
54198,Slipstream (1973),drama,"popular, but mysterious rock d.j mike mallard...",8


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# For class weight adjustment due to class imbalance problem
labels = processed_train_data['labels'].values
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(processed_train_data)
# test_dataset = Dataset.from_pandas(processed_test_data)

# 3. Tokenization

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenizer_function, batched=True)
# tokenized_test_dataset = test_dataset.map(tokenizer_function, batched=True)

Map:   0%|          | 0/54214 [00:00<?, ? examples/s]

Map:   0%|          | 0/54200 [00:00<?, ? examples/s]

In [ ]:
df_tk = pd.DataFrame(tokenized_train_dataset)

train_df, test_df = train_test_split(df_tk, test_size=0.2, stratify=df_tk['labels'], random_state=42, shuffle=True)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
train = Dataset.from_pandas(train_df)
test = Dataset.from_pandas(test_df)

# 4. Create a BERT model

In [ ]:
config = AutoConfig.from_pretrained("distilbert-base-uncased")
config.hidden_dropout_prob = 0.4
config.num_labels = count_label

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    config=config
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 5. Train the model

In [ ]:
training_args = TrainingArguments(
    output_dir=result_dir,
    learning_rate=7e-5,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    num_train_epochs=5,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    # train_dataset=tokenized_train_dataset,
    # eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    class_weights=class_weights_tensor
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(test)
pred_labels = np.argmax(predictions.predictions, axis=-1)

conf_matrix = confusion_matrix(predictions.label_ids, pred_labels)

In [ ]:
plt.figure(figsize=(6, 4))
unique_labels = processed_train_data[['Genre', 'labels']].drop_duplicates()
sorted_labels = unique_labels.sort_values(by='labels')
original_labels = sorted_labels['Genre'].tolist()

sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=True,
            xticklabels=original_labels, yticklabels=original_labels)

plt.xticks(fontsize=8, rotation=45, ha='right')
plt.yticks(fontsize=8)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix Heatmap")
plt.show()

In [ ]:
trainer.evaluate()